In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import Basemap
import os
os.environ["PROJ_LIB"] = "C:\\Users\\Mike\\Anaconda3\\Library\\share\\"; #fixr
from mpl_toolkits.basemap import Basemap
from netCDF4 import Dataset
import netCDF4
from matplotlib.backends.backend_pdf import PdfPages
from matplotlib.colors import BoundaryNorm
from matplotlib.ticker import MaxNLocator

# Defines shaded 
levs = np.arange(100,1200,100)
levs = np.append([-1,0,1,10,50],levs)
cmap = plt.get_cmap('terrain')
norm = BoundaryNorm(levs, ncolors=cmap.N, clip=True)

path_to_netcdf     = "C:/Users/Mike/Desktop/Offshore/Topo_NETCDF/"
Path_to_shapefiles = "C:/Users/Mike/Desktop/Offshore/Boundaries/"
#########################################################################################################
### Terrain data from NETCDF3 grid 
#########################################################################################################
# Parent domain d01 .nc file [NOAA Weather and Climate Toolkit]
fh = Dataset(path_to_netcdf+'wrfout_d01_2017-10-28_06-var69-t0.nc', mode='r')
lons_d01           = fh.variables['lon'][:]
lats_d01           = fh.variables['lat'][:]
Terrain_HGT_d01    = fh.variables['HGT'][0,:,:]
lon_d01,lat_d01    = np.meshgrid(lons_d01, lats_d01)
fh.close()
# Nest d02 .nc file [NOAA Weather and Climate Toolkit]
fh = Dataset(path_to_netcdf+'wrfout_d02_2017-10-28_06-var69-t0.nc', mode='r')
lons_d02           = fh.variables['lon'][:]
lats_d02           = fh.variables['lat'][:]
Terrain_HGT_d02    = fh.variables['HGT'][0,:,:]
lon_d02,lat_d02    = np.meshgrid(lons_d02, lats_d02)
fh.close()
# Nest d03 .nc file [NOAA Weather and Climate Toolkit]
fh = Dataset(path_to_netcdf+'wrfout_d03_2017-10-28_06-var69-t0.nc', mode='r')
lons_d03           = fh.variables['lon'][:]
lats_d03           = fh.variables['lat'][:]
Terrain_HGT_d03    = fh.variables['HGT'][0,:,:]
lon_d03,lat_d03    = np.meshgrid(lons_d03, lats_d03)
fh.close()
# Nest d04 .nc file [NOAA Weather and Climate Toolkit]
fh = Dataset(path_to_netcdf+'wrfout_d04_2017-10-28_06-var69-t0.nc', mode='r')
lons_d04           = fh.variables['lon'][:]
lats_d04           = fh.variables['lat'][:]
Terrain_HGT_d04    = fh.variables['HGT'][0,:,:]
lon_d04,lat_d04    = np.meshgrid(lons_d04, lats_d04)
fh.close()
#########################################################################################################
### Bounds for image
#########################################################################################################
# ***This needs to be adjusted manually***
Buffer_inc = -3 #degrees
North      = np.max(lats_d01)+Buffer_inc+0.30
East       = np.max(lons_d01)+Buffer_inc-1.00
South      = np.min(lats_d01)-Buffer_inc-0.50
West       = np.min(lons_d01)-Buffer_inc+2.00
#########################################################################################################
### Basemap projection information
#########################################################################################################
def plot_conus():
    """ Create a plot of the Continential US. """
    m = Basemap(
        lat_1=South,
        lat_2=North,
        lat_0=East,
        lon_0=West,
        llcrnrlon=West,
        llcrnrlat=South,
        urcrnrlon=East,
        urcrnrlat=North,
        projection='lcc',
        resolution='f') #resolution is set to 'full'
    m.drawcoastlines(linewidth=0.5,zorder=6)
    m.drawcountries(linewidth=0.5,zorder=6)
    m.drawstates(linewidth=0.5,zorder=6)
    return m
#########################################################################################################
### Plotting figure
#########################################################################################################
fig = plt.figure(figsize=(10, 10))
m = plot_conus()
#########
### Shapefile boundary information [Optional... These files were pre-processed using ArcMapv10.2]
#########
### d01
m.readshapefile(Path_to_shapefiles+"d01", 'Dissolve_INT', drawbounds = False)
m.readshapefile(Path_to_shapefiles+"d01", 'd01', drawbounds = False)
for info, shape in zip(m.d01_info, m.d01):
    d01x, d01y = zip(*shape) 
m.plot(d01x, d01y, marker=None,ls='-.',color='k',linewidth=3.0,zorder=8)   
### d02
m.readshapefile(Path_to_shapefiles+"d02", 'd02', drawbounds = False)
for info, shape in zip(m.d02_info, m.d02):
    d02x, d02y = zip(*shape) 
m.plot(d02x, d02y, marker=None,ls='-.',color='k',linewidth=2.5,zorder=8)
### d03
m.readshapefile(Path_to_shapefiles+"d03", 'd03', drawbounds = False)
for info, shape in zip(m.d03_info, m.d03):
    d03x, d03y = zip(*shape) 
m.plot(d03x, d03y, marker=None,ls='-' ,color='K',linewidth=2.0,zorder=8)
### d04
m.readshapefile(Path_to_shapefiles+"d04", 'd04', drawbounds = False)
for info, shape in zip(m.d04_info, m.d04):
    d04x, d04y = zip(*shape) 
m.plot(d04x, d04y, marker=None,ls='-' ,color='K',linewidth=2.0,zorder=8)
#########
### HGT pcolormesh information
#########
### d01 | cmap='nipy_spectral' looks good too
Terrain_HGT_d01[Terrain_HGT_d01<0] = np.nan
cax = m.pcolormesh(lon_d01,lat_d01,Terrain_HGT_d01,latlon=True,cmap=cmap, norm=norm,zorder=2,rasterized=True)
cax = m.pcolormesh(lon_d02,lat_d02,Terrain_HGT_d02,latlon=True,cmap=cmap, norm=norm,zorder=3,rasterized=True)
cax = m.pcolormesh(lon_d03,lat_d03,Terrain_HGT_d03,latlon=True,cmap=cmap, norm=norm,zorder=4,rasterized=True)
cax = m.pcolormesh(lon_d04,lat_d04,Terrain_HGT_d04,latlon=True,cmap=cmap, norm=norm,zorder=5,rasterized=True)
cb  = m.colorbar(cax,pad=0,label="Height [m]")
cb.set_ticks([-1,0,1,10,50,100,200,300,400,500,600,700,800,900,1000,1100])
#########
### Stations within boundary information [Optional... This file was pre-processed using ArcMapv10.2]
#########
Stations = pd.read_csv(r'C:/Users/Mike/Desktop/Offshore/Case_Studies/Stations_Within_Domain.csv')
x, y = m(np.asarray(Stations['Lon']),np.asarray(Stations['Lat']))
m.scatter(x,y,s=15,marker='o',color='r',zorder=7)
#########
### Title
#########
plt.title('Domain Configuration, Modeled Terrain Height, ''\n''& Surface Stations for Analysis',fontsize = 20)

#########
### Saving image
#########
#.png
plt.savefig("Domain_config.png")
#.pdf
pp = PdfPages('Domain_config.pdf')
pp.savefig(fig)
pp.close()

C:\Users\Mike\Anaconda3\lib\site-packages\ipykernel_launcher.py:76: MatplotlibDeprecationWarning: 
The dedent function was deprecated in Matplotlib 3.1 and will be removed in 3.3. Use inspect.cleandoc instead.
